In [1]:
import json
import pandas as pd
from sql_queries import ValorantQueries

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [6]:
# Get score table where round_number == 1
score_table = vq.get_full_round()
score_table_df = pd.DataFrame(score_table, columns=["RoundID", "MatchID", "RoundNumber", "Attacking_Team", "WinningTeam", "Team_A_Economy", "Team_B_Economy", "Win_condition", "Ceremony"])
score_table_df = score_table_df[score_table_df["RoundNumber"] == 5]
attacking_team = int(score_table_df["Attacking_Team"])
winning_team = int(score_table_df["WinningTeam"])
winner = 1 if attacking_team == winning_team else 0
winner

1

In [3]:
vq.get_full_round()

[(695324, 43621, 1, 53, 53, 0, 0, 'kills', 'flawless'),
 (695325, 43621, 2, 53, 53, 3, 1, 'kills', 'team_ace'),
 (695326, 43621, 3, 53, 53, 3, 4, 'bomb', 'closer'),
 (695327, 43621, 4, 53, 53, 4, 3, 'bomb', 'default'),
 (695328, 43621, 5, 53, 53, 4, 4, 'kills', 'default'),
 (695329, 43621, 6, 53, 444, 4, 2, 'defuse', 'thrifty'),
 (695330, 43621, 7, 53, 444, 4, 4, 'defuse', 'default'),
 (695331, 43621, 8, 53, 444, 4, 4, 'defuse', 'default'),
 (695332, 43621, 9, 53, 53, 3, 4, 'kills', 'default'),
 (695333, 43621, 10, 53, 444, 4, 1, 'defuse', 'thrifty'),
 (695334, 43621, 11, 53, 444, 3, 4, 'defuse', 'default'),
 (695335, 43621, 12, 53, 444, 4, 4, 'time', 'default'),
 (695336, 43621, 13, 444, 444, 0, 0, 'kills', 'default'),
 (695337, 43621, 14, 444, 444, 1, 3, 'kills', 'default'),
 (695338, 43621, 15, 444, 53, 4, 3, 'kills', 'default'),
 (695339, 43621, 16, 444, 53, 4, 2, 'kills', 'flawless'),
 (695340, 43621, 17, 444, 53, 4, 4, 'time', 'default'),
 (695341, 43621, 18, 444, 53, 4, 3, 'defu

In [ ]:
score_table

### AAA